In [158]:
import pandas as pd 
import numpy as np 
import matplotlib_inline 
import matplotlib.pyplot as plt 

test_set=pd.read_csv('test.csv') 
train_set=pd.read_csv('train.csv') 

In [159]:
print(test_set.head())
print(test_set.info())  


   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       ------------

In [160]:
train_corr=train_set.select_dtypes(include=[np.number]) 
corr_matrix=train_corr.corr() 
print(corr_matrix["Survived"].sort_values(ascending=False)) 

Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Name: Survived, dtype: float64


In [161]:
train_set_labels=train_set["Survived"]

In [162]:
from sklearn.impute import SimpleImputer 
imputer=SimpleImputer(strategy="median") 
train_num=train_set.drop(["Survived", "Sex", "Embarked", "Ticket", "Name", "Cabin"], axis=1) 
imputer.fit(train_num)  
print(imputer.statistics_)
x=imputer.transform(train_num)  
train_num_tr=pd.DataFrame(x, columns=train_num.columns, index=train_num.index) 
print(train_num_tr) 


[446.       3.      28.       0.       0.      14.4542]
     PassengerId  Pclass   Age  SibSp  Parch     Fare
0            1.0     3.0  22.0    1.0    0.0   7.2500
1            2.0     1.0  38.0    1.0    0.0  71.2833
2            3.0     3.0  26.0    0.0    0.0   7.9250
3            4.0     1.0  35.0    1.0    0.0  53.1000
4            5.0     3.0  35.0    0.0    0.0   8.0500
..           ...     ...   ...    ...    ...      ...
886        887.0     2.0  27.0    0.0    0.0  13.0000
887        888.0     1.0  19.0    0.0    0.0  30.0000
888        889.0     3.0  28.0    1.0    2.0  23.4500
889        890.0     1.0  26.0    0.0    0.0  30.0000
890        891.0     3.0  32.0    0.0    0.0   7.7500

[891 rows x 6 columns]


In [163]:
from sklearn.preprocessing import OrdinalEncoder 
oe=OrdinalEncoder() 
train_set["Embarked"].fillna(train_set["Embarked"].mode()[0], inplace=True)
train_cat=train_set[["Sex", "Embarked"]] 
train_cat_en=oe.fit_transform(train_cat) 
print(train_cat_en[:40]) 
train_cat_tr=pd.DataFrame(train_cat_en, columns=["Sex", "Embarked"], index=train_set.index) 
print(train_cat_tr) 

[[1. 2.]
 [0. 0.]
 [0. 2.]
 [0. 2.]
 [1. 2.]
 [1. 1.]
 [1. 2.]
 [1. 2.]
 [0. 2.]
 [0. 0.]
 [0. 2.]
 [0. 2.]
 [1. 2.]
 [1. 2.]
 [0. 2.]
 [0. 2.]
 [1. 1.]
 [1. 2.]
 [0. 2.]
 [0. 0.]
 [1. 2.]
 [1. 2.]
 [0. 1.]
 [1. 2.]
 [0. 2.]
 [0. 2.]
 [1. 0.]
 [1. 2.]
 [0. 1.]
 [1. 2.]
 [1. 0.]
 [0. 0.]
 [0. 1.]
 [1. 2.]
 [1. 0.]
 [1. 2.]
 [1. 0.]
 [1. 2.]
 [0. 2.]
 [0. 0.]]
     Sex  Embarked
0    1.0       2.0
1    0.0       0.0
2    0.0       2.0
3    0.0       2.0
4    1.0       2.0
..   ...       ...
886  1.0       2.0
887  0.0       2.0
888  0.0       2.0
889  1.0       0.0
890  1.0       1.0

[891 rows x 2 columns]


C:\Users\prtku\AppData\Local\Temp\ipykernel_24268\1969216547.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_set["Embarked"].fillna(train_set["Embarked"].mode()[0], inplace=True)


In [164]:
train_set_prepared=pd.concat([train_num_tr, train_cat_tr], axis=1) 
print(train_set_prepared.head(10)) 

   PassengerId  Pclass   Age  SibSp  Parch     Fare  Sex  Embarked
0          1.0     3.0  22.0    1.0    0.0   7.2500  1.0       2.0
1          2.0     1.0  38.0    1.0    0.0  71.2833  0.0       0.0
2          3.0     3.0  26.0    0.0    0.0   7.9250  0.0       2.0
3          4.0     1.0  35.0    1.0    0.0  53.1000  0.0       2.0
4          5.0     3.0  35.0    0.0    0.0   8.0500  1.0       2.0
5          6.0     3.0  28.0    0.0    0.0   8.4583  1.0       1.0
6          7.0     1.0  54.0    0.0    0.0  51.8625  1.0       2.0
7          8.0     3.0   2.0    3.0    1.0  21.0750  1.0       2.0
8          9.0     3.0  27.0    0.0    2.0  11.1333  0.0       2.0
9         10.0     2.0  14.0    1.0    0.0  30.0708  0.0       0.0


In [165]:
from xgboost import XGBClassifier 
from sklearn.model_selection import cross_val_predict 
xgbc = XGBClassifier(n_estimators=210, learning_rate=0.01, max_depth=9, subsample=0.6, colsample_bytree=0.8, random_state=42) 
xgbc_pred = cross_val_predict(xgbc, train_set_prepared, train_set_labels, cv=5)


In [166]:
from sklearn.model_selection import GridSearchCV 
param_grid={ 'n_estimators': [120, 125, 130], 'max_depth': [4, 5, 6] } 
search=GridSearchCV(estimator=xgbc, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1) 
search.fit(train_set_prepared, train_set_labels) 
print("best parameters: ", search.best_params_) 
print("best score: ", search.best_score_) 

Fitting 5 folds for each of 9 candidates, totalling 45 fits
best parameters:  {'max_depth': 6, 'n_estimators': 120}
best score:  0.8148703785073129


In [167]:
from lightgbm import LGBMClassifier 
lgbm=LGBMClassifier(n_estimators=100, learning_rate=0.01, max_depth=5, random_state=42, force_row_wise = True) 
lgbm.fit(train_set_prepared, train_set_labels) 
lgbm_pred=cross_val_predict(lgbm, train_set_prepared, train_set_labels, cv=5) 
print("predictions= ", lgbm_pred[:20]) 
print("labels= ", list(train_set_labels[:20]))  

[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Total Bins 476
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [168]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
stack = StackingClassifier(estimators=[('lgbm', lgbm), ('xgb', xgbc) ], final_estimator=LogisticRegression() ) 
stack.fit(train_set_prepared, train_set_labels)
stack_pred = cross_val_predict(stack, train_set_prepared, train_set_labels, cv=5)


[LightGBM] [Info] Number of positive: 342, number of negative: 549
[LightGBM] [Info] Total Bins 476
[LightGBM] [Info] Number of data points in the train set: 891, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383838 -> initscore=-0.473288
[LightGBM] [Info] Start training from score -0.473288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

In [169]:
from sklearn.metrics import accuracy_score 
acc=accuracy_score(train_set_labels, lgbm_pred)  
print(acc) 
acc3 = accuracy_score(train_set_labels, xgbc_pred) 
print(acc3) 
acc5 = accuracy_score(train_set_labels, stack_pred) 
print(acc5) 

0.819304152637486
0.8271604938271605
0.8294051627384961


In [170]:
test_corr=test_set.select_dtypes(include=[np.number]) 
corr2_matrix=test_corr.corr() 


In [171]:
test_num=test_set.drop(["Sex", "Embarked", "Ticket", "Name", "Cabin"], axis=1) 
print(imputer.statistics_)
y=imputer.transform(test_num)  
test_num_tr=pd.DataFrame(y, columns=test_num.columns, index=test_num.index) 
print(test_num_tr) 

[446.       3.      28.       0.       0.      14.4542]
     PassengerId  Pclass   Age  SibSp  Parch      Fare
0          892.0     3.0  34.5    0.0    0.0    7.8292
1          893.0     3.0  47.0    1.0    0.0    7.0000
2          894.0     2.0  62.0    0.0    0.0    9.6875
3          895.0     3.0  27.0    0.0    0.0    8.6625
4          896.0     3.0  22.0    1.0    1.0   12.2875
..           ...     ...   ...    ...    ...       ...
413       1305.0     3.0  28.0    0.0    0.0    8.0500
414       1306.0     1.0  39.0    0.0    0.0  108.9000
415       1307.0     3.0  38.5    0.0    0.0    7.2500
416       1308.0     3.0  28.0    0.0    0.0    8.0500
417       1309.0     3.0  28.0    1.0    1.0   22.3583

[418 rows x 6 columns]


In [172]:
test_set["Embarked"].fillna(test_set["Embarked"].mode()[0], inplace=True)
test_cat=test_set[["Sex", "Embarked"]] 
test_cat_en=oe.transform(test_cat) 
print(test_cat_en[:40]) 
test_cat_tr=pd.DataFrame(test_cat_en, columns=["Sex", "Embarked"], index=test_set.index) 
print(test_cat_tr) 

[[1. 1.]
 [0. 2.]
 [1. 1.]
 [1. 2.]
 [0. 2.]
 [1. 2.]
 [0. 1.]
 [1. 2.]
 [0. 0.]
 [1. 2.]
 [1. 2.]
 [1. 2.]
 [0. 2.]
 [1. 2.]
 [0. 2.]
 [0. 0.]
 [1. 1.]
 [1. 0.]
 [0. 2.]
 [0. 0.]
 [1. 0.]
 [1. 2.]
 [0. 2.]
 [1. 0.]
 [0. 0.]
 [1. 2.]
 [0. 0.]
 [1. 0.]
 [1. 2.]
 [1. 0.]
 [1. 2.]
 [1. 2.]
 [0. 2.]
 [0. 2.]
 [1. 0.]
 [1. 0.]
 [0. 2.]
 [0. 2.]
 [1. 2.]
 [1. 2.]]
     Sex  Embarked
0    1.0       1.0
1    0.0       2.0
2    1.0       1.0
3    1.0       2.0
4    0.0       2.0
..   ...       ...
413  1.0       2.0
414  0.0       0.0
415  1.0       2.0
416  1.0       2.0
417  1.0       0.0

[418 rows x 2 columns]


C:\Users\prtku\AppData\Local\Temp\ipykernel_24268\162822803.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  test_set["Embarked"].fillna(test_set["Embarked"].mode()[0], inplace=True)


In [173]:
test_set_prepared=pd.concat([test_num_tr, test_cat_tr], axis=1) 
print(test_set_prepared.head(10)) 

   PassengerId  Pclass   Age  SibSp  Parch     Fare  Sex  Embarked
0        892.0     3.0  34.5    0.0    0.0   7.8292  1.0       1.0
1        893.0     3.0  47.0    1.0    0.0   7.0000  0.0       2.0
2        894.0     2.0  62.0    0.0    0.0   9.6875  1.0       1.0
3        895.0     3.0  27.0    0.0    0.0   8.6625  1.0       2.0
4        896.0     3.0  22.0    1.0    1.0  12.2875  0.0       2.0
5        897.0     3.0  14.0    0.0    0.0   9.2250  1.0       2.0
6        898.0     3.0  30.0    0.0    0.0   7.6292  0.0       1.0
7        899.0     2.0  26.0    1.0    1.0  29.0000  1.0       2.0
8        900.0     3.0  18.0    0.0    0.0   7.2292  0.0       0.0
9        901.0     3.0  21.0    2.0    0.0  24.1500  1.0       2.0


In [174]:
test_pred=stack.predict(test_set_prepared)  

In [175]:
final=pd.DataFrame( {"PassengerId": test_set["PassengerId"], "Survived": test_pred} ) 
final.to_csv("titanic_final_submission.csv", index=False) 